## Example - Flowing cold ion collision integral test
This example tests whether the cold ion collision integral reproduces the expected analytical value in the low flow limit.

This notebook is the v2.0.0 rewrite of the notebook that corresponds to the third test in Section 5.2.2. of the ReMKiT1D code paper.

**NOTE**: This now also has non-unity densities and temperatures to check that the v2.0.0 bug fix works so it is slightly different to the paper version.

In [ ]:
import numpy as np
import holoviews as hv
import matplotlib.pyplot as plt

import RMK_support as rmk
import RMK_support.common_models as cm

### Some useful constants

In [ ]:
elCharge = 1.60218e-19
elMass = 9.10938e-31
amu = 1.6605390666e-27  # atomic mass unit
ionMass = 2.014*amu  # deuterium mass
epsilon0 = 8.854188e-12  # vacuum permittivity
heavySpeciesMass = 2.014  # in amus


### Context initialization

In [ ]:
rk = rmk.RMKContext()
rk.IOContext = rmk.IOContext(HDF5Dir="./RMKOutput/RMK_CI_test/")
rk.mpiContext = rmk.MPIContext(4)

### Grid setup

In [ ]:
xGrid = 150*np.ones(64)
dv0 = 0.0307
cv = 1.025
vGrid = [dv0]
for i in range(1,120):
    vGrid.append(vGrid[i-1]*cv)
lMax = 1
rk.grid = rmk.Grid(xGrid, np.array(vGrid), lMax, interpretXGridAsWidths=True, interpretVGridAsWidths=True, isPeriodic=True)
L = sum(xGrid)

### Set default species and temperature derivations

In [ ]:
rk.textbook = rmk.Textbook(rk.grid,[-1,0]) 

rk.species.add(rmk.Species("e",0))
rk.species.add(rmk.Species("D+",-1,atomicA=2.014,charge=1.0))


### Variables

In [ ]:
nInit = 2*np.ones(rk.grid.numX)
uInit = 0.0001*np.ones(rk.grid.numX) # Slow flow in order to reproduce the analytical solution.

TInit = 1.5*np.ones(rk.grid.numX)
WInit = 3*nInit*TInit/2
fInit = np.zeros([rk.grid.numX,rk.grid.numH,rk.grid.numV])
for i in range(rk.grid.numX):
    fInit[i,rk.grid.getH(0)-1,:] = (np.pi*TInit[i])**(-1.5) * nInit[i]* np.exp(-rk.grid.vGrid**2/TInit[i])

# Rescale distribution function to ensure that the numerical density moment agrees with the initial values
numerical_dens = rk.grid.velocityMoment(fInit,0,1)
for i in range(rk.grid.numX):
    fInit[i,rk.grid.getH(0)-1,:] = nInit[i] *fInit[i,rk.grid.getH(0)-1,:]/numerical_dens[i]
    
f,f_dual = rmk.varAndDual("f",rk.grid,isDistribution=True,data=fInit)
W = rmk.Variable("W",rk.grid,derivation=rk.textbook["energyMoment"],derivationArgs=["f"])
n,n_dual = rmk.varAndDual("n",rk.grid,derivation=rk.textbook["densityMoment"],derivationArgs=["f"])
ni,ni_dual = rmk.varAndDual("ni",rk.grid,data=nInit)
Gi_dual, Gi = rmk.varAndDual("Gi",rk.grid,primaryOnDualGrid=True,data=nInit*uInit)
ui_dual,ui = rmk.varAndDual("ui",rk.grid,primaryOnDualGrid=True,derivation=rmk.derivations.NodeDerivation("ui",rmk.node(Gi_dual)/rmk.node(ni_dual))) 

G_dual,G = rmk.varAndDual("G",rk.grid,primaryOnDualGrid=True,derivation=rk.textbook["fluxMoment"],derivationArgs=["f"])
u_dual,u = rmk.varAndDual("u",rk.grid,primaryOnDualGrid=True,derivation=rmk.derivations.NodeDerivation("u",rmk.node(G_dual)/rmk.node(n_dual))) 
T,T_dual = rmk.varAndDual("T",rk.grid,derivation=rk.textbook["tempFromEnergye"],derivationArgs=["W","n","zeroVar"])
zeroVar = rmk.Variable("zeroVar",rk.grid,isDerived=True,inOutput=False)

rk.variables.add(f,f_dual,W,n,n_dual,ni,ni_dual,Gi_dual,Gi,ui_dual,ui,G_dual,G,u_dual,u,T,T_dual,zeroVar)


## Adding the flowing cold ion electron-ion model

In [ ]:
rk.models.add(cm.flowingIonEIColl(rk.grid,rk.textbook,rk.norms,f,ni,ui_dual,n_dual,T_dual,rk.species["D+"],list(range(2,rk.grid.numH+1)),Gi_dual).rename("e-i_l>0"))

### Integrator and timestep options

Simple single step backwards Euler integration

In [ ]:
integrator = rmk.BDEIntegrator("BDE",absTol=10.0,nonlinTol=1e-14,convergenceVars=[f,Gi_dual])
integrationStep = rmk.IntegrationStep("BE",integrator)
integrationStep.add(rk.models) 
rk.integrationScheme = rmk.IntegrationScheme(dt=0.1,steps=integrationStep) 
rk.integrationScheme.setFixedNumTimesteps(1000,100) 

#### Generate a LaTeX summary of the ReMKiT1D run 

In [ ]:
rk.generatePDF("Flowing e-i collision test")

### Create config file

In [ ]:
rk.writeConfigFile()


### Data analysis


#### Loading data

In [ ]:
loadedData = rk.loadSimulation()
dataset=loadedData.dataset

In [ ]:
hv.extension('matplotlib')
%matplotlib inline
plt.rcParams['figure.dpi'] = 150
hv.output(size=80, dpi=150)


### Relative velocity error

In [ ]:
abs(dataset["u"].data[-1,0]-dataset["ui"].data[-1,0])/dataset["ui"].data[-1,0]

### Compare with analytic solution

In [ ]:
f_analytic = np.zeros([rk.grid.numX,rk.grid.numV])
final_ue=dataset["u"].data[-1,:]
for i in range(rk.grid.numX):
    f_analytic[i,:] = 2*final_ue[i]*rk.grid.vGrid *(np.pi*TInit[i])**(-1.5) * nInit[i]* np.exp(-rk.grid.vGrid**2/TInit[i])/TInit[i]

In [ ]:
dataName = 'f'
curve = hv.Curve((rk.grid.vGrid,f_analytic[0,:]),label='Analytical formula') *hv.Scatter((rk.grid.vGrid,loadedData["f"].data[-1,0,1,:]),label='Numerical result').opts(s=6.0,color='r',marker='x')
curve.opts(xlabel='v [$v_{th}$]',ylabel='f$_1$ [normalized units]')

### Relative total momentum error

In [ ]:
initMom = dataset["G_dual"][0,0]*elMass/ionMass +dataset["Gi_dual"][0,0]   # initial momentum in electron units
endMom = dataset["G_dual"][:,0]*elMass/ionMass +dataset["Gi_dual"][:,0]  # final momentum in electron units

momErr=abs(endMom-initMom)/initMom

In [ ]:
momErr

In [ ]:
hv.output(fig='pdf')
hv.save(curve.opts(xlabel='v [$v_{th}$]',ylabel='f$_1$ [normalized units]'), 'e-if1Relaxation.pdf', dpi=144)